<a href="https://colab.research.google.com/github/vatsal-ts/BTP_MQTT/blob/main/mqtt_prelims.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install scikit-learn
!pip install pandas
!pip install keras
!pip install tensorflow
!pip install xgboost
!pip install scikeras

In [ ]:
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn import tree
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as pyplot
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import ConfusionMatrixDisplay
# from sklearn.metrics import plot_confusion_matrix
from warnings import simplefilter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/mqttset.zip -d /content/mqttset

Archive:  /content/drive/MyDrive/mqttset.zip
  inflating: /content/mqttset/Data/CSV/bruteforce.csv  
  inflating: /content/mqttset/Data/CSV/flood.csv  
  inflating: /content/mqttset/Data/CSV/legitimate_1w.csv  
  inflating: /content/mqttset/Data/CSV/malaria.csv  
  inflating: /content/mqttset/Data/CSV/malformed.csv  
  inflating: /content/mqttset/Data/CSV/slowite.csv  
  inflating: /content/mqttset/Data/FINAL_CSV/mqttdataset_reduced.csv  
  inflating: /content/mqttset/Data/FINAL_CSV/test30.csv  
  inflating: /content/mqttset/Data/FINAL_CSV/test30_augmented.csv  
  inflating: /content/mqttset/Data/FINAL_CSV/test30_reduced.csv  
  inflating: /content/mqttset/Data/FINAL_CSV/train70.csv  
  inflating: /content/mqttset/Data/FINAL_CSV/train70_augmented.csv  
  inflating: /content/mqttset/Data/FINAL_CSV/train70_reduced.csv  
  inflating: /content/mqttset/Data/PCAP/bruteforce.pcapng  
  inflating: /content/mqttset/Data/PCAP/capture_1w.pcap  
  inflating: /content/mqttset/Data/PCAP/capture_floo

In [ ]:
import time
start = time.time()

#import two dataset splitted
dftrain = pd.read_csv("/content/mqttset/Data/FINAL_CSV/train70_reduced.csv")
dftest = pd.read_csv("/content/mqttset/Data/FINAL_CSV/test30_reduced.csv")
# Transpose the describe result to make columns as rows
describe_train = dftrain.describe().transpose()
describe_test = dftest.describe().transpose()

# Filter columns to drop in dftrain
columns_to_drop_train = describe_train[(describe_train['min'] == 0) & (describe_train['max'] == 0)].index.tolist()

# Filter columns to drop in dftest
columns_to_drop_test = describe_test[(describe_test['min'] == 0) & (describe_test['max'] == 0)].index.tolist()

# Drop columns from dftrain and dftest
dftrain = dftrain.drop(columns=columns_to_drop_train)
dftest = dftest.drop(columns=columns_to_drop_test)
simplefilter(action='ignore', category=FutureWarning)
seed = 7

#train
#print(dftrain.loc[dftrain['target'] == 'legitimate'])
class_names = dftrain.target.unique()
dftrain=dftrain.astype('category')
cat_columns = dftrain.select_dtypes(['category']).columns
dftrain[cat_columns] = dftrain[cat_columns].apply(lambda x: x.cat.codes)
#print(dftrain.loc[125, 'target'])
x_columns = dftrain.columns.drop('target')
x_train = dftrain[x_columns].values
y_train = dftrain['target']
x_train=x_train[:-1]
y_train=y_train[:-1]


#test
class_names = dftest.target.unique()
dftest=dftest.astype('category')
cat_columns = dftest.select_dtypes(['category']).columns
dftest[cat_columns] = dftest[cat_columns].apply(lambda x: x.cat.codes)
x_columns = dftest.columns.drop('target')
x_test = dftest[x_columns].values
y_test = dftest['target']


print("Ready to generate train and test datasets")

Ready to generate train and test datasets


In [ ]:
#Neural network
print("Starting Random forest")
model = Sequential()
model.add(Dense(50, input_dim=x_train.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(30, input_dim=x_train.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(20, kernel_initializer='normal'))
model.add(Dense(6,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
history = model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=200,batch_size=1000)
end = time.time()
diff=end-start
print("Training time: " + str(diff))
starttest = time.time()
y_pred_nn = model.predict(x_test)
y_pred_nn = np.argmax(y_pred_nn,axis=1)
endtest =time.time()
difftest = endtest-starttest
print("Test time: " + str(difftest))

print(model.summary())


#Create Naive Bayes Classifier
print("Starting Naive Bayes")
gnb = GaussianNB()
gnb.fit(x_train, y_train)
end = time.time()
diff=end-start
print("Training time: " + str(diff))
starttest = time.time()
y_pred_nb = gnb.predict(x_test)
endtest =time.time()
difftest = endtest-starttest
print("Test time: " + str(difftest))

#Decision tree
print("Starting Decision tree")
clf = DecisionTreeClassifier()
clf = clf.fit(x_train,y_train)
end = time.time()
diff=end-start
print("Training time: " + str(diff))
starttest = time.time()
y_pred_dt = clf.predict(x_test)
y_pred_dt_roc = clf.predict_proba(x_test)
endtest =time.time()
difftest = endtest-starttest
print("Test time: " + str(difftest))


print("Decision Tree, accuracy: " + str(metrics.accuracy_score(y_test, y_pred_dt)) + " F1 score:" + str(metrics.f1_score(y_test, y_pred_dt,average='weighted')))
matrixdt = confusion_matrix(y_test,y_pred_dt)
print(matrixdt)


print("Naive Bayes, accuracy: " + str(metrics.accuracy_score(y_test, y_pred_nb)) + " F1 score:" + str(metrics.f1_score(y_test, y_pred_nb,average='weighted')))
matrixnv = confusion_matrix(y_test,y_pred_nb)
print(matrixnv)


print("Neural network, accuracy: " + str(metrics.accuracy_score(y_test, y_pred_nn)) + " F1 score:" + str(metrics.f1_score(y_test, y_pred_nn,average='weighted')))
matrixnn = confusion_matrix(y_test,y_pred_nn)
print(matrixnn)

Starting Random forest
Epoch 1/200
232/232 - 3s - loss: 1.1654 - accuracy: 0.7599 - val_loss: 0.5814 - val_accuracy: 0.7804 - 3s/epoch - 14ms/step
Epoch 2/200
232/232 - 2s - loss: 0.4979 - accuracy: 0.8068 - val_loss: 0.4769 - val_accuracy: 0.8155 - 2s/epoch - 7ms/step
Epoch 3/200
232/232 - 3s - loss: 0.3994 - accuracy: 0.8700 - val_loss: 0.3984 - val_accuracy: 0.8625 - 3s/epoch - 11ms/step
Epoch 4/200
232/232 - 2s - loss: 0.3571 - accuracy: 0.8835 - val_loss: 0.3900 - val_accuracy: 0.8624 - 2s/epoch - 8ms/step
Epoch 5/200
232/232 - 2s - loss: 0.3175 - accuracy: 0.8910 - val_loss: 0.3775 - val_accuracy: 0.8613 - 2s/epoch - 9ms/step
Epoch 6/200
232/232 - 2s - loss: 0.2676 - accuracy: 0.9064 - val_loss: 0.4437 - val_accuracy: 0.8424 - 2s/epoch - 8ms/step
Epoch 7/200
232/232 - 1s - loss: 0.2469 - accuracy: 0.9152 - val_loss: 0.4471 - val_accuracy: 0.8565 - 980ms/epoch - 4ms/step
Epoch 8/200
232/232 - 1s - loss: 0.2312 - accuracy: 0.9176 - val_loss: 0.5833 - val_accuracy: 0.8456 - 965ms/ep

In [ ]:
from sklearn.svm import SVC

# Create and train an SVM classifier
print("Starting Support Vector Machine (SVM)")
svm_classifier = SVC(kernel='poly')  # You can choose different kernels (e.g., 'linear', 'rbf', 'poly', etc.)
svm_classifier.fit(x_train, y_train)

# Predict using the SVM classifier
y_pred_svm = svm_classifier.predict(x_test)

# Evaluate the SVM model
print("Support Vector Machine (SVM), accuracy:", metrics.accuracy_score(y_test, y_pred_svm))
print("F1 score:", metrics.f1_score(y_test, y_pred_svm, average='weighted'))
matrix_svm = confusion_matrix(y_test, y_pred_svm)
print(matrix_svm)

Starting Support Vector Machine (SVM)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Create and train a k-NN classifier
print("Starting k-Nearest Neighbors (k-NN)")
knn_classifier = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors (k) as needed
knn_classifier.fit(x_train, y_train)

# Predict using the k-NN classifier
y_pred_knn = knn_classifier.predict(x_test)

# Evaluate the k-NN model
print("k-Nearest Neighbors (k-NN), accuracy:", metrics.accuracy_score(y_test, y_pred_knn))
print("F1 score:", metrics.f1_score(y_test, y_pred_knn, average='weighted'))
matrix_knn = confusion_matrix(y_test, y_pred_knn)
print(matrix_knn)

Starting k-Nearest Neighbors (k-NN)


KeyboardInterrupt: ignored

In [ ]:
from sklearn.linear_model import LogisticRegression

# Create and train a logistic regression classifier
print("Starting Logistic Regression")
logistic_classifier = LogisticRegression(solver='liblinear')  # You can choose different solvers based on your data
logistic_classifier.fit(x_train, y_train)

# Predict using the logistic regression classifier
y_pred_logistic = logistic_classifier.predict(x_test)

# Evaluate the logistic regression model
print("Logistic Regression, accuracy:", metrics.accuracy_score(y_test, y_pred_logistic))
print("F1 score:", metrics.f1_score(y_test, y_pred_logistic, average='weighted'))
matrix_logistic = confusion_matrix(y_test, y_pred_logistic)
print(matrix_logistic)

Starting Logistic Regression
Logistic Regression, accuracy: 0.7939772383925874
F1 score: 0.7630931146718104
[[  423   425     0  3502     1     0]
 [    0 27851     0 11226     0     0]
 [    1     2    87    94     0     0]
 [    0     0     0 49639     0     0]
 [    1   283     0  2935    33    26]
 [    0   115     0  1845     0   801]]


In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost import XGBClassifier  # If you're using XGBoost for classification
from xgboost import XGBRegressor   # If you're using XGBoost for regression

# Create and train additional classifiers
xgb_classifier= XGBClassifier()
rf_classifier = RandomForestClassifier()
lr_classifier = LogisticRegression()

xgb_classifier.fit(x_train, y_train)
rf_classifier.fit(x_train, y_train)
lr_classifier.fit(x_train, y_train)

# Suppose you have already trained your XGBoost classifier (xgb_classifier)

# Create a VotingClassifier with majority voting
ensemble_model = VotingClassifier(estimators=[
    ('XGBoost', xgb_classifier),
    ('Random Forest', rf_classifier),
    ('Logistic Regression', lr_classifier)
], voting='hard')

# Fit the ensemble model on the individual models' predictions
ensemble_model.fit(np.array([xgb_classifier.predict(x_test), rf_classifier.predict(x_test), lr_classifier.predict(x_test)]).T, y_test)

# Make predictions with the ensemble model
y_pred_ensemble = ensemble_model.predict(np.array([xgb_classifier.predict(x_test), rf_classifier.predict(x_test), lr_classifier.predict(x_test)]).T)

# Evaluate the ensemble model
print("Ensemble model, accuracy:", metrics.accuracy_score(y_test, y_pred_ensemble))
print("F1 score:", metrics.f1_score(y_test, y_pred_ensemble, average='weighted'))
matrix_ensemble = confusion_matrix(y_test, y_pred_ensemble)
print(matrix_ensemble)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Ensemble model, accuracy: 0.9047033940980965
F1 score: 0.9023862131065937
[[ 3521   484     0     7   339     0]
 [  232 35555     0  3250    40     0]
 [    2     4    89    88     1     0]
 [   49  3122     0 46468     0     0]
 [ 1102   276     7   459  1434     0]
 [    0     0     0     0     0  2761]]


In [ ]:
import numpy as np
import time
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.callbacks import EarlyStopping
from sklearn import metrics
from sklearn.metrics import confusion_matrix

# Assuming you have loaded and preprocessed your data into x_train, y_train, x_test, y_test

# Create a Sequential model
model = Sequential()

# Add a 1D convolutional layer with 32 filters and kernel size 3
model.add(Conv1D(32, kernel_size=3, input_shape=(x_train.shape[1], 1), activation='relu'))

# Add a 1D MaxPooling layer with pool size 2
model.add(MaxPooling1D(pool_size=2))

# Flatten the output of the convolutional layer
model.add(Flatten())

# Add a fully connected layer with 128 units and ReLU activation
model.add(Dense(128, activation='relu'))

# Add the output layer with the number of classes and softmax activation
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define an early stopping callback
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

# Train the model
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), callbacks=[monitor], verbose=2, epochs=200, batch_size=1000)

# Measure training time
end = time.time()
diff = end - start
print("Training time: " + str(diff))

# Make predictions on the test data
starttest = time.time()
y_pred_cnn = model.predict(x_test)
y_pred_cnn = np.argmax(y_pred_cnn, axis=1)
endtest = time.time()
difftest = endtest - starttest
print("Test time: " + str(difftest))

# Print model summary
print(model.summary())

# Evaluate the model
print("CNN, accuracy: " + str(metrics.accuracy_score(y_test, y_pred_cnn)) + " F1 score:" + str(metrics.f1_score(y_test, y_pred_cnn, average='weighted')))
matrix_cnn = confusion_matrix(y_test, y_pred_cnn)
print(matrix_cnn)


Epoch 1/200
232/232 - 9s - loss: 6.9204 - accuracy: 0.7693 - val_loss: 2.7288 - val_accuracy: 0.8104 - 9s/epoch - 39ms/step
Epoch 2/200
232/232 - 1s - loss: 1.6503 - accuracy: 0.8612 - val_loss: 2.2658 - val_accuracy: 0.8487 - 1s/epoch - 4ms/step
Epoch 3/200
232/232 - 1s - loss: 1.2311 - accuracy: 0.8821 - val_loss: 2.0652 - val_accuracy: 0.8541 - 996ms/epoch - 4ms/step
Epoch 4/200
232/232 - 1s - loss: 1.3121 - accuracy: 0.8914 - val_loss: 1.7354 - val_accuracy: 0.8784 - 1s/epoch - 5ms/step
Epoch 5/200
232/232 - 1s - loss: 0.7808 - accuracy: 0.9051 - val_loss: 1.7923 - val_accuracy: 0.8051 - 1s/epoch - 5ms/step
Epoch 6/200
232/232 - 1s - loss: 0.6118 - accuracy: 0.9085 - val_loss: 1.5959 - val_accuracy: 0.8690 - 997ms/epoch - 4ms/step
Epoch 7/200
232/232 - 1s - loss: 0.6621 - accuracy: 0.9118 - val_loss: 1.2639 - val_accuracy: 0.8907 - 981ms/epoch - 4ms/step
Epoch 8/200
232/232 - 1s - loss: 1.4230 - accuracy: 0.8915 - val_loss: 1.9212 - val_accuracy: 0.8358 - 1s/epoch - 5ms/step
Epoch 

In [ ]:
import numpy as np
import time
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping
from sklearn import metrics
from sklearn.metrics import confusion_matrix

# Assuming you have loaded and preprocessed your data into x_train, y_train, x_test, y_test

# Create a Sequential model
model = Sequential()

# Add a 1D convolutional layer with 64 filters and kernel size 3
model.add(Conv1D(64, kernel_size=3, input_shape=(x_train.shape[1], 1), activation='relu'))

# Add a 1D MaxPooling layer with pool size 2
model.add(MaxPooling1D(pool_size=2))

# Add another convolutional layer with 128 filters and kernel size 3
model.add(Conv1D(128, kernel_size=3, activation='relu'))

# Add another MaxPooling layer with pool size 2
model.add(MaxPooling1D(pool_size=2))

# Flatten the output of the convolutional layer
model.add(Flatten())

# Add a fully connected layer with 256 units and ReLU activation
model.add(Dense(256, activation='relu'))

# Add Dropout for regularization
model.add(Dropout(0.5))

# Add the output layer with the number of classes and softmax activation
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define an early stopping callback
# monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

# Train the model
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), callbacks=[monitor], verbose=2, epochs=100, batch_size=1000)

# Measure training time
end = time.time()
diff = end - start
print("Training time: " + str(diff))

# Make predictions on the test data
starttest = time.time()
y_pred_cnn = model.predict(x_test)
y_pred_cnn = np.argmax(y_pred_cnn, axis=1)
endtest = time.time()
difftest = endtest - starttest
print("Test time: " + str(difftest))

# Print model summary
print(model.summary())

# Evaluate the model
print("CNN, accuracy: " + str(metrics.accuracy_score(y_test, y_pred_cnn)) + " F1 score:" + str(metrics.f1_score(y_test, y_pred_cnn, average='weighted')))
matrix_cnn = confusion_matrix(y_test, y_pred_cnn)
print(matrix_cnn)


Epoch 1/100
232/232 - 4s - loss: 4.6628 - accuracy: 0.7602 - val_loss: 0.4837 - val_accuracy: 0.8360 - 4s/epoch - 17ms/step
Epoch 2/100
232/232 - 2s - loss: 0.4909 - accuracy: 0.8467 - val_loss: 0.3759 - val_accuracy: 0.8733 - 2s/epoch - 8ms/step
Epoch 3/100
232/232 - 1s - loss: 0.3729 - accuracy: 0.8880 - val_loss: 0.3399 - val_accuracy: 0.8823 - 1s/epoch - 5ms/step
Epoch 4/100
232/232 - 2s - loss: 0.3284 - accuracy: 0.8996 - val_loss: 0.3493 - val_accuracy: 0.8823 - 2s/epoch - 9ms/step
Epoch 5/100
232/232 - 2s - loss: 0.3057 - accuracy: 0.9047 - val_loss: 0.3374 - val_accuracy: 0.8893 - 2s/epoch - 9ms/step
Epoch 6/100
232/232 - 2s - loss: 0.2871 - accuracy: 0.9090 - val_loss: 0.3531 - val_accuracy: 0.8785 - 2s/epoch - 10ms/step
Epoch 7/100
232/232 - 2s - loss: 0.2741 - accuracy: 0.9112 - val_loss: 0.3543 - val_accuracy: 0.8569 - 2s/epoch - 9ms/step
Epoch 8/100
232/232 - 1s - loss: 0.2743 - accuracy: 0.9115 - val_loss: 0.3477 - val_accuracy: 0.8576 - 1s/epoch - 5ms/step
Epoch 9/100
23

In [ ]:
import numpy as np
import time
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
from sklearn import metrics
from sklearn.metrics import confusion_matrix

# Assuming you have loaded and preprocessed your data into x_train, y_train, x_test, y_test

# Create a Sequential model
model = Sequential()

# Add a 1D convolutional layer with 64 filters and kernel size 3
model.add(Conv1D(64, kernel_size=3, input_shape=(x_train.shape[1], 1), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))

# Add another convolutional layer with 128 filters and kernel size 3
model.add(Conv1D(128, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))

# Add another convolutional layer with 256 filters and kernel size 3
model.add(Conv1D(256, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))

# Flatten the output of the convolutional layer
model.add(Flatten())

# Add a fully connected layer with 512 units and ReLU activation
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

# Add another fully connected layer with 256 units and ReLU activation
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# Add the output layer with the number of classes and softmax activation
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define an early stopping callback
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

# Train the model
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), callbacks=[monitor], verbose=2, epochs=200, batch_size=1000)

# Measure training time
end = time.time()
diff = end - start
print("Training time: " + str(diff))

# Make predictions on the test data
starttest = time.time()
y_pred_cnn = model.predict(x_test)
y_pred_cnn = np.argmax(y_pred_cnn, axis=1)
endtest = time.time()
difftest = endtest - starttest
print("Test time: " + str(difftest))

# Print model summary
print(model.summary())

# Evaluate the model
print("CNN, accuracy: " + str(metrics.accuracy_score(y_test, y_pred_cnn)) + " F1 score:" + str(metrics.f1_score(y_test, y_pred_cnn, average='weighted')))
matrix_cnn = confusion_matrix(y_test, y_pred_cnn)
print(matrix_cnn)




Epoch 1/200
232/232 - 6s - loss: 0.5503 - accuracy: 0.7895 - val_loss: 0.6503 - val_accuracy: 0.7844 - 6s/epoch - 27ms/step
Epoch 2/200
232/232 - 2s - loss: 0.4566 - accuracy: 0.8205 - val_loss: 0.6514 - val_accuracy: 0.7701 - 2s/epoch - 11ms/step
Epoch 3/200
232/232 - 2s - loss: 0.3683 - accuracy: 0.8585 - val_loss: 2.0179 - val_accuracy: 0.4078 - 2s/epoch - 9ms/step
Epoch 4/200
232/232 - 2s - loss: 0.2741 - accuracy: 0.9022 - val_loss: 2.5272 - val_accuracy: 0.3944 - 2s/epoch - 9ms/step
Epoch 5/200
232/232 - 2s - loss: 0.2635 - accuracy: 0.9056 - val_loss: 2.4899 - val_accuracy: 0.5261 - 2s/epoch - 9ms/step
Epoch 6/200
232/232 - 2s - loss: 0.2493 - accuracy: 0.9108 - val_loss: 3.1107 - val_accuracy: 0.5336 - 2s/epoch - 9ms/step
Epoch 6: early stopping
Training time: 1353.0507462024689
3103/3103 [==============================] - 11s 3ms/step
Test time: 20.685484409332275
Model: "sequential_10"
_________________________________________________________________
 Layer (type)            